In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib.request

Get links to every team's page

In [2]:
%%time
def get_teams(soup):
    '''Takes BeautifulSoup object of NBA teams page and returns
       list of links to team pages.'''
   
    for link in soup.find_all('a'):
        team = link.text
        if 'teams' in link.get('href'):
            yield link.get('href')
        if team == 'Washington Wizards':
            break

site = 'https://www.basketball-reference.com/teams/'
page= urllib.request.urlopen(site)
soup = BeautifulSoup(page, 'lxml')

teamlinks = list(get_teams(soup))[1:]

Wall time: 1.41 s


From teamlinks, get every rostered player in league.

In [3]:
%%time
def get_players(teams, year):
    '''From list of teams, get list of all players on that 
       team for a given year. This is only set up to work
       on seasons from 2009-10 on. There may be errors going
       further back.'''
    
    year = str(year)
    
    for team in teams:
        link = 'https://www.basketball-reference.com' + team + year + '.html'
        if 'NJN' in link:
            if year > '2012':
                link = 'https://www.basketball-reference.com/teams/BRK/' + year + '.html'
            else:
                link = 'https://www.basketball-reference.com/teams/NJN/' + year + '.html'
        if 'CHA' in link:
                if year > '2014':
                    link = 'https://www.basketball-reference.com/teams/CHO/' + year + '.html'
                elif year < '2003':
                    link = 'https://www.basketball-reference.com/teams/CHH/' + year + '.html'
                else:
                    'https://www.basketball-reference.com/teams/CHA/'+ year + '.html'
                
        if 'NOH' in link:
            if year > '2013':
                link = 'https://www.basketball-reference.com/teams/NOP/' + year + '.html'
            else:
                 link = 'https://www.basketball-reference.com/teams/NOH/' + year + '.html'
               
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'lxml')

        for i in soup.find_all('table')[0].find_all('a'):
            if 'players' in i['href']:
                yield (i.text, i['href'])

Wall time: 0 ns


In [4]:
%%time
player_set = set()
for year in range(2010,2020):
    new_set = set(get_players(teamlinks,year))
    if len(player_set) == 0:
        player_set = new_set
    else:
        player_set.update(new_set)
    print(year, len(player_set))

2010 442
2011 523
2012 611
2013 692
2014 770
2015 852
2016 926
2017 1014
2018 1133
2019 1238
Wall time: 7min 9s


Get stats for the last season for each player

In [5]:
%%time
all_season_df = pd.DataFrame()
x=1
for player in player_set:
    reference_site = 'https://www.basketball-reference.com' + player[1]
    page = urllib.request.urlopen(reference_site)
    soup = BeautifulSoup(page, 'lxml')
    per_game_table = soup.table
    
    # Excludes rookies from the table
    if per_game_table != None:
        height = str(soup.find_all('div', {'id':'info'})[0].find_all('span', {'itemprop':'height'})[0].string)
        height = height.split('-')
        height = int(height[0])*12 + int(height[1])
        player_df = pd.read_html(reference_site)[0]
        player_df['Height'] = height
        player_df['href'] = player[1]
        player_df['Player'] = player[0]
        
        # removes career totals
        player_df = player_df[player_df.Season != 'Career']
        # removes seasons where players did not play
        player_df = player_df[player_df.Tm != player_df.Lg]
        # remove totals for each team player played on
        player_df = player_df[player_df.Age.notna()]
        # trim season stat to be the starting year
        player_df = plyaer_df.Season.str[:4]
        # removes seasons before 2009-10
        player_df = player_df[player_df['Season'] > '2008']
        all_season_df = all_season_df.append(player_df)
        print(x, player_df.Player.unique())
        x += 1

1 ['Jeff Green']
2 ['JaJuan Johnson']
3 ['Kevin Love']
4 ['Kelly Olynyk']
5 ['Malik Monk']
6 ['Andrei Kirilenko']
7 ["DeAndre' Bembry"]
8 ['Billy Garrett']
9 ['Andre Iguodala']
10 ['Kemba Walker']
11 ['Seth Curry']
12 ['Malik Hairston']
13 ['Andrew Wiggins']
14 ['Marquis Daniels']
15 ['Troy Murphy']
16 ['Greg Monroe']
17 ['Nolan Smith']
18 ['David Nwaba']
19 ['Shannon Brown']
20 ['Rodions Kurucs']
21 ['Maurice Ndour']
22 ['Jack Cooley']
23 ['Kevin Huerter']
24 ['Jeff Adrien']
25 ['Brad Miller']
26 ['Andre Miller']
27 ['Andrew Goudelock']
28 ['Bryn Forbes']
29 ['Thomas Welsh']
30 ['Omri Casspi']
31 ['Vernon Macklin']
32 ['Thomas Robinson']
33 ['Tim Quarterman']
34 ['Cedi Osman']
35 ['Keith Bogans']
36 ['Josh Childress']
37 ['Travis Diener']
38 ['Kirk Hinrich']
39 ['A.J. Hammons']
40 ['Samardo Samuels']
41 ["Jermaine O'Neal"]
42 ['Montrezl Harrell']
43 ['Solomon Alabi']
44 ['Bobby Portis']
45 ["D'Angelo Russell"]
46 ['Solomon Hill']
47 ['Clint Capela']
48 ['Thon Maker']
49 ['Earl Boykins

379 ['Kyle Collinsworth']
380 ['Brandon Jennings']
381 ['Toney Douglas']
382 ['Viacheslav Kravtsov']
383 ['Doug McDermott']
384 ['Mitch McGary']
385 ['Spencer Hawes']
386 ['Jon Brockman']
387 ['Ime Udoka']
388 ['Amile Jefferson']
389 ['Malachi Richardson']
390 ['Shaquille Harrison']
391 ['Carldell Johnson']
392 ['D.J. White']
393 ['Chris Wright']
394 ['Gary Clark']
395 ['Kyrie Irving']
396 ['Chris Babb']
397 ['Mickaël Piétrus']
398 ['Chandler Hutchison']
399 ['Marco Belinelli']
400 ['Garrett Temple']
401 ['Rasho Nesterović']
402 ['Carrick Felix']
403 ['Andrew Nicholson']
404 ['DaJuan Summers']
405 ['Craig Smith']
406 ['Ben Hansbrough']
407 ['Rasual Butler']
408 ['Michael Gbinije']
409 ['Carlos Boozer']
410 ['David Wear']
411 ['Ricky Davis']
412 ['Lonzo Ball']
413 ['Walt Lemon']
414 ['Derrick Caracter']
415 ['Al Horford']
416 ['Marc Gasol']
417 ['Luol Deng']
418 ['Hedo Türkoğlu']
419 ['Paul George']
420 ['Mo Williams']
421 ['Will Bynum']
422 ['Alonzo Gee']
423 ['Myke Henry']
424 ['Elija

752 ['Roy Hibbert']
753 ['Bernard James']
754 ['Brendan Haywood']
755 ['Georges Niang']
756 ['Sebastian Telfair']
757 ['George King']
758 ['Etan Thomas']
759 ['Nemanja Bjelica']
760 ['Pape Sy']
761 ['Langston Galloway']
762 ['Markieff Morris']
763 ['Johnathan Motley']
764 ['Zabian Dowdell']
765 ['Shaun Livingston']
766 ['Robbie Hummel']
767 ['Tim Hardaway']
768 ['Eric Bledsoe']
769 ['Hakim Warrick']
770 ['Elfrid Payton']
771 ['Oleksiy Pecherov']
772 ['Kris Joseph']
773 ['Sundiata Gaines']
774 ['Willy Hernangómez']
775 ['Tyler Davis']
776 ['Jason Thompson']
777 ['P.J. Hairston']
778 ['Larry Hughes']
779 ['Lindsey Hunter']
780 ['Orien Greene']
781 ['Dragan Bender']
782 ['Mike James']
783 ['Ben Moore']
784 ['Kris Humphries']
785 ['Dewayne Dedmon']
786 ['Kurt Thomas']
787 ['Danny Granger']
788 ['Henry Walker']
789 ['Chimezie Metu']
790 ['James White']
791 ['Andre Ingram']
792 ['John Holland']
793 ['Rudy Gay']
794 ['Aaron Gray']
795 ['Tim Thomas']
796 ['Drew Eubanks']
797 ['Gal Mekel']
798 

1121 ['John Collins']
1122 ['Brandon Bass']
1123 ['Donald Sloan']
1124 ['Trey Lyles']
1125 ['Mitch Creek']
1126 ['Al Harrington']
1127 ['Manny Harris']
1128 ['Ramon Sessions']
1129 ['Lucas Nogueira']
1130 ['JaKarr Sampson']
1131 ['Jordan Williams']
1132 ['Kendall Marshall']
1133 ['Donté Greene']
1134 ['Julian Washburn']
1135 ['Channing Frye']
1136 ['Jarvis Varnado']
1137 ['Jameel Warney']
1138 ['Shelden Williams']
1139 ['Troy Caupain']
1140 ['Eric Gordon']
1141 ['Chris Johnson']
1142 ['Alfonzo McKinnie']
1143 ['Dennis Smith']
1144 ['Zach Collins']
1145 ['David West']
1146 ['Kostas Papanikolaou']
1147 ['Brandan Wright']
1148 ['Gilbert Arenas']
1149 ['Marcus Williams']
1150 ['J.J. Hickson']
1151 ['Jacob Wiley']
1152 ['Mikal Bridges']
1153 ['Scotty Hopson']
1154 ['Erick Dampier']
1155 ['Juwan Howard']
1156 ['Mike Miller']
1157 ['Coby Karl']
1158 ['Kyle Kuzma']
1159 ['Hilton Armstrong']
1160 ['Richaun Holmes']
1161 ['C.J. Williams']
1162 ['Trey Gilder']
1163 ['Josh Davis']
1164 ['John Jenk

In [6]:
all_season_df.to_csv('all_season.csv')